# Query SQL Server in a container

This notebook:

1. Downloads a container image. The container contains an instance of SQL Server with a sample database installed.
2. Starts the container in docker on the local machine.
3. Connects to the SQL Server instance and queries a database
4. Stops the docker container.
5. Removes the container image from docker. 

## Prerequisites

The notebook is designed to run on Windows in Azure Data Studio.

* Add `pyodbc` package. Click Manage Packages above this page. 
* Add ODBC Driver 17 for SQL Server to the computer that is running this notebook.

## Configure docker environment
Make sure you have your docker environment configured, more information [here](https://docs.docker.com/get-started/). 
> NOTE
> <br>If you are using Docker Desktop for Windows or Mac, the default configuration will limit your containers to 2 cores, regardless of the number of cores on your computer. Be sure to configure docker to allow at least 4 cores and 4GB of RAM for this demo to run properly. To do this, right click on the Docker Desktop icon in the status bar and choose Settings -> Advanced.

The docker commands may take some time to execute, but you will not see progress here until they are complete.



## Pull the demo container

In [1]:
! docker pull bluefooted/sql2019tempdbdemo

Using default tag: latest
latest: Pulling from bluefooted/sql2019tempdbdemo
Digest: sha256:035a1bda5539bfe68ad1b2f032a6e389cea91a0cd880e75b83ef186c46b2e34f
Status: Image is up to date for bluefooted/sql2019tempdbdemo:latest
docker.io/bluefooted/sql2019tempdbdemo:latest


## Start the demo container

In [2]:
! docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=P@ssw0rd!" -p 1455:1433 --name sql2019tempdbdemo -d bluefooted/sql2019tempdbdemo

fbea7ca9ce3264284d714e03aa091b1e023a7c3472b1cf45253e6d10f35295ac


## Connect & query 

### Set environment variables

To connect to SQL Server, you need to import `pyodbc`, and set the server, database, username, and password.

> NOTE
> This example uses a hard-coded password. This is a terrible idea.

The container is running on local host, but it uses a non-default TCP port of `1455`. This was set at the `docker run` command previously. 

You could also connect to the SQL Server with Azure Data Studio, sqlcmd, or SSMS using the values set here:

   ```
   server = 'localhost,1455'
   database = 'Adventureworks'
   username = 'sa'
   password = 'P@ssw0rd!'
   ```



In [3]:
import pyodbc
server = 'localhost,1455'
database = 'Adventureworks'
username = 'sa'
password = 'P@ssw0rd!'

### Set connection string

With the environment variables set, you can now set the connections string. 

In [4]:
#Connection String
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = connection.cursor()

### Query SQL Server

The following query returns a list of every person in the sample database with a last name begins with "Ti".

This notebook is running a python kernel, so you need to wrap the T-SQL statement in Python. The actual T-SQL command is 

```sql
SELECT DISTINCT FirstName + ' ' + LastName FROM Person.Person WHERE LastName LIKE 'Ti%';
```

In [5]:
#Sample select query
cursor.execute("SELECT DISTINCT FirstName + ' ' + LastName FROM Person.Person WHERE LastName LIKE 'Ti%';")
row = cursor.fetchone()
while row:
    print (row[0])
    row = cursor.fetchone()

Danielle Tiedt
Diane Tibbott
Hung-Fu Ting
John Tippett
Mike Tiano
Tony Ting
Yuping Tian


## Stop SQL Server instance

In [6]:
! docker stop sql2019tempdbdemo

sql2019tempdbdemo


## Remove container

In [7]:
! docker rm sql2019tempdbdemo

sql2019tempdbdemo
